In [19]:
    # import tracemalloc; tracemalloc.start(); import linecache

    # snapshot = tracemalloc.take_snapshot()
    # display_top(snapshot)

def display_top(snapshot, key_type='lineno', limit=3):
    snapshot = snapshot.filter_traces((
        tracemalloc.Filter(False, "<frozen importlib._bootstrap>"),
        tracemalloc.Filter(False, "<unknown>"),
    ))
    top_stats = snapshot.statistics(key_type)

    print("Top %s lines" % limit)
    for index, stat in enumerate(top_stats[:limit], 1):
        frame = stat.traceback[0]
        # replace "/path/to/module/file.py" with "module/file.py"
        filename = os.sep.join(frame.filename.split(os.sep)[-2:])
        print("#%s: %s:%s: %.1f KiB"
              % (index, filename, frame.lineno, stat.size / 1024))
        line = linecache.getline(frame.filename, frame.lineno).strip()
        if line:
            print('    %s' % line)

    other = top_stats[limit:]
    if other:
        size = sum(stat.size for stat in other)
        print("%s other: %.1f KiB" % (len(other), size / 1024))
    total = sum(stat.size for stat in top_stats)
    print("Total allocated size: %.1f KiB" % (total / 1024))

In [4]:
import torch
import numpy as np
x = torch.zeros(50,50)

np.random.randint(low=0,high=50, size=(50,50))

array([[23, 42, 34, ..., 24, 11,  2],
       [25, 33, 35, ..., 48, 34, 41],
       [45, 30, 26, ..., 20, 11, 30],
       ...,
       [ 3,  2,  2, ..., 26, 36, 23],
       [20,  5, 23, ..., 30, 40, 31],
       [ 8, 43, 28, ..., 21, 46,  8]])

In [13]:
from UTILS.tensor_ops import my_view, __hash__, objload

obs_feed_new, prev_obs_feed, valid_mask, N_valid, next_his_pool = objload()
__hash__(roll_hisory(obs_feed_new, prev_obs_feed, valid_mask, N_valid, next_his_pool))
%timeit roll_hisory(obs_feed_new, prev_obs_feed, valid_mask, N_valid, next_his_pool)

2.05 ms ± 3.93 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
n_timestep = reward_logits.shape[0]
n_agent = reward_logits.shape[1]
assert not ((reward != 0) & (reward!=1) & (reward!=-0.5) & (reward!=0.5)).any()
# {-0.5, 0, 0.5, 1} -> {3<(-1), 0, 1, 2}
reward_projection = lambda r: np.where(r<0, 3, r*2).astype(np.int)
# apply projection
reward_class = reward_projection(reward) # np_one_hot(,4)
# ($time_step.$n_agent.) -> ($time_step.$n_agent_.$n_agent.)
reward_class = np_repeat_at(reward_class, insert_dim=1, n_times=n_agent)
# not zero reward
not_zero_mask = (reward_class!=0)
# reward_class_onehot
reward_class_onehot = np_one_hot(reward_class, 4)
# reward estimation
reward_logits = np_softmax(reward_logits, axis=-1)
prediction_rating = np.take_along_axis(reward_logits, axis=-1, 
    indices=np.expand_dims(reward_class, -1)).squeeze(-1)
correct_prediction = (np.argmax(reward_logits, -1) == reward_class)

In [ ]:
confidence_threshhold = 0.25*1.05
eye_mat = np_repeat_at(np.eye(n_agent), insert_dim=0, n_times=n_timestep)
compasion_matrix = correct_prediction & (prediction_rating > confidence_threshhold) & not_zero_mask | (eye_mat==1)
print(compasion_matrix)

In [ ]:
# compasion_matrix = (prediction_rating > confidence_threshhold) & not_zero_mask & (eye_mat==0)
compasion_matrix = compasion_matrix.astype(np.int)

# 计算分解后的reward，因为reward可能被多个智能体分解
n_com = compasion_matrix.sum(-2)
assert not (n_com==0).any()
reward_decmp = np.where(n_com!=0, reward/n_com, reward) # reward/(n_com+1e-7)
# https://numpy.org/doc/stable/reference/generated/numpy.matmul.html
compassion_reward = np.matmul(compasion_matrix, np.expand_dims(reward_decmp,-1)).squeeze(-1)
print(compassion_reward)

In [ ]:
'''
函数说明：在有限的、不均衡的多标签数据集中，按照预设的比例，取出尽可能多的样本
'''
def sample_balance(x, y, n_class, weight=None):
    if weight is None: weight = torch.ones(n_class, device=x.device)
    else: weight = torch.Tensor(weight, device=x.device)
    n_instance = torch.zeros(n_class, device=x.device)
    indices = [None]*n_class
    for i in range(n_class):
        indices[i] = torch.where(y==i)[0]
        n_instance[i] = len(indices[i])
    ratio = n_instance/weight
    bottle_neck = torch.argmin(n_instance/weight)
    r = ratio[bottle_neck]
    n_sample = (r*weight).long()
    # print(n_instance, n_sample)
    new_indices = [indices[i][torch.randperm(n_sample[i])] for i in range(n_class)]
    # print(new_indices)
    new_indices_ = torch.cat(new_indices)
    assert len(new_indices_) == sum(n_sample)
    return x[new_indices_], y[new_indices_]

'''
测试代码
'''
x = torch.rand(200, 4)
y1 = torch.rand(100, 4)
y2 = torch.rand(100, 4)
y2[:, 0] += 1
y = torch.cat((y1,y2))
y = torch.argmax(y, -1)
print(y)
n_class = 4
weight = [2,1,1,1]
sample_balance(x,y,4,weight)

In [ ]:
x = torch.rand(100, 4)
y = torch.rand(100, 4)
y = torch.argmax(y, -1)

# return x[new_indices_], y[new_indices_]

In [ ]:
import numpy as np
def my_view(x, shape):
    if -1 in shape[1:-1]: return my_view_test(x, shape)
    reverse_lookup = True if shape[0] == -1 else False
    if not reverse_lookup:
        for i, dim in enumerate(shape):
            if dim == 0:
                shape[i] = x.shape[i]
    else:
        for i in range(len(shape)):
            ni = -(i + 1)  # iter -1,-2,-3,...
            dim = shape[ni]
            if dim == 0:
                shape[ni] = x.shape[ni]
    if isinstance(x, np.ndarray):
        return x.reshape(*shape)
    return x.view(*shape)

def my_view_test(x, shape):
    # fill both way until meet -1 
    for i, dim in enumerate(shape):
        if dim == 0: shape[i] = x.shape[i]
        elif dim == -1: break
    for i in range(len(shape)):
        ni = -(i + 1); dim = shape[ni]
        if dim == 0: shape[ni] = x.shape[ni]
        elif dim == -1: break
    # print(shape)
    if isinstance(x, np.ndarray):
        return x.reshape(*shape)
    return x.view(*shape)



In [ ]:

"""
    improve np.reshape and torch.view function
    If a dim is assigned with 0, it will keep its original dimension
    eg.1    x.shape = (4, 5, 6, 7); new_shape = [0, 0, -1]
            y = my_view(x, new_shape)
            y.shape = (4, 5, 6*7)

    eg.2    x.shape = (4, 5, 6, 7); new_shape = [-1, 0, 0]
            y = my_view(x, new_shape)
            y.shape = (4*5, 6, 7)

(4, 5, 6); new_shape = [0, 0, -1, 3]

(4, 5, 6); new_shape = [2, -1, 0, 0]
(3, 4, 5, 6); new_shape = [0, 2, -1, 0, 0]
"""

my_view(np.zeros(shape=(4, 5, 6)), shape = [2, -1, 0, 0]).shape

In [ ]:
my_view(np.zeros(shape=(4, 5, 6)), shape = [0, 0, -1, 3]).shape

In [ ]:
load_ext Cython

In [ ]:
%%cython

a: cython.int = 0
for i in range(10):
    a += i
print(a)

In [ ]:
%%cython



In [ ]:
new_method(np.ones(5), np.zeros(5))